# SelfQueryRetriever로 메타데이터 기반 자동 필터링

`SelfQueryRetriever`는 자연어 질의를 구조화된 쿼리로 자동 변환하여 메타데이터 기반 필터링을 수행하는 강력한 검색 도구입니다.

## 주요 특징
- 자연어 질의를 구조화된 쿼리로 자동 변환
- 메타데이터 필터 자동 추출 및 적용
- 의미적 유사성 검색과 메타데이터 필터링 결합
- 다양한 벡터 데이터베이스 지원

## 작동 원리
1. 사용자의 자연어 질의 입력
2. Query-constructing LLM chain이 구조화된 쿼리 생성
3. 구조화된 쿼리를 VectorStore에 적용
4. 필터링된 결과 반환

[참고] LangChain이 지원하는 셀프 쿼리 검색기 목록: https://python.langchain.com/docs/integrations/retrievers/self_query

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

## 1. 샘플 데이터 및 벡터 저장소 생성

화장품 상품 정보를 담은 문서와 메타데이터를 준비하고, 벡터 저장소를 구축합니다.

In [2]:
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

# 화장품 상품의 설명과 메타데이터 생성
docs = [
    Document(
        page_content="수분 가득한 히알루론산 세럼으로 피부 속 깊은 곳까지 수분을 공급합니다.",
        metadata={"year": 2024, "category": "스킨케어", "user_rating": 4.7},
    ),
    Document(
        page_content="24시간 지속되는 매트한 피니시의 파운데이션, 모공을 커버하고 자연스러운 피부 표현이 가능합니다.",
        metadata={"year": 2023, "category": "메이크업", "user_rating": 4.5},
    ),
    Document(
        page_content="식물성 성분으로 만든 저자극 클렌징 오일, 메이크업과 노폐물을 부드럽게 제거합니다.",
        metadata={"year": 2023, "category": "클렌징", "user_rating": 4.8},
    ),
    Document(
        page_content="비타민 C 함유 브라이트닝 크림, 칙칙한 피부톤을 환하게 밝혀줍니다.",
        metadata={"year": 2023, "category": "스킨케어", "user_rating": 4.6},
    ),
    Document(
        page_content="롱래스팅 립스틱, 선명한 발색과 촉촉한 사용감으로 하루종일 편안하게 사용 가능합니다.",
        metadata={"year": 2024, "category": "메이크업", "user_rating": 4.4},
    ),
    Document(
        page_content="자외선 차단 기능이 있는 톤업 선크림, SPF50+/PA++++ 높은 자외선 차단 지수로 피부를 보호합니다.",
        metadata={"year": 2024, "category": "선케어", "user_rating": 4.9},
    ),
]

# 벡터 저장소 생성
vectorstore = Chroma.from_documents(
    docs, OpenAIEmbeddings(model="text-embedding-3-small")
)

## 2. 메타데이터 필드 정의

`AttributeInfo` 클래스를 사용하여 각 메타데이터 필드의 정보를 정의합니다.
이 정보는 LLM이 자연어 질의에서 적절한 필터를 추출하는 데 사용됩니다.

In [3]:
from langchain.chains.query_constructor.base import AttributeInfo

# 메타데이터 필드 정보 생성
metadata_field_info = [
    AttributeInfo(
        name="category",
        description="The category of the cosmetic product. One of ['스킨케어', '메이크업', '클렌징', '선케어']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the cosmetic product was released",
        type="integer",
    ),
    AttributeInfo(
        name="user_rating",
        description="A user rating for the cosmetic product, ranging from 1 to 5",
        type="float",
    ),
]

## 3. SelfQueryRetriever 생성

`SelfQueryRetriever.from_llm()` 메서드를 사용하여 retriever를 생성합니다.

In [4]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# SelfQueryRetriever 생성
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="Brief summary of a cosmetic product",
    metadata_field_info=metadata_field_info,
)

## 4. 기본 Query 테스트

자연어 질의를 통해 메타데이터 필터링이 자동으로 수행되는 것을 확인합니다.

In [5]:
# 평점 기반 필터링
results = retriever.invoke("평점이 4.8 이상인 제품을 추천해주세요")
print("평점 4.8 이상 제품:")
for doc in results:
    print(f"- {doc.page_content[:30]}... ({doc.metadata['category']}, 평점: {doc.metadata['user_rating']})")

평점 4.8 이상 제품:
- 자외선 차단 기능이 있는 톤업 선크림 (선케어, 평점: 4.9)
- 식물성 성분으로 만든 저자극 클렌징 오일 (클렌징, 평점: 4.8)


In [6]:
# 연도 기반 필터링
results = retriever.invoke("2023년에 출시된 상품을 추천해주세요")
print("\n2023년 출시 제품:")
for doc in results:
    print(f"- {doc.page_content[:30]}... ({doc.metadata['category']}, {doc.metadata['year']}년)")


2023년 출시 제품:
- 24시간 지속되는 매트한 피니시의 파운데이션 (메이크업, 2023년)
- 비타민 C 함유 브라이트닝 크림 (스킨케어, 2023년)
- 식물성 성분으로 만든 저자극 클렌징 오일 (클렌징, 2023년)


In [7]:
# 카테고리 기반 필터링
results = retriever.invoke("카테고리가 선케어인 상품을 추천해주세요")
print("\n선케어 카테고리 제품:")
for doc in results:
    print(f"- {doc.page_content[:30]}... ({doc.metadata['category']})")


선케어 카테고리 제품:
- 자외선 차단 기능이 있는 톤업 선크림 (선케어)


In [8]:
# 복합 조건 필터링
results = retriever.invoke(
    "카테고리가 메이크업인 상품 중에서 평점이 4.5 이상인 상품을 추천해주세요"
)
print("\n복합 조건 검색 결과:")
for doc in results:
    print(f"- {doc.page_content[:30]}... ({doc.metadata['category']}, 평점: {doc.metadata['user_rating']})")


복합 조건 검색 결과:
- 24시간 지속되는 매트한 피니시의 파운데이션 (메이크업, 평점: 4.5)


## 5. 검색 결과 개수 제한

`enable_limit=True`를 설정하면 검색 결과 개수를 제한할 수 있습니다.

In [9]:
# 검색 결과 개수 제한 설정
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="Brief summary of a cosmetic product",
    metadata_field_info=metadata_field_info,
    enable_limit=True,  # 검색 결과 제한 기능 활성화
    search_kwargs={"k": 2},  # 최대 2개 결과 반환
)

results = retriever.invoke("2023년에 출시된 상품을 추천해주세요")
print("검색 결과 (최대 2개):")
for doc in results:
    print(f"- {doc.page_content[:30]}... ({doc.metadata['year']}년)")

검색 결과 (최대 2개):
- 24시간 지속되는 매트한 피니시의 파운데이션 (2023년)
- 비타민 C 함유 브라이트닝 크림 (2023년)


In [10]:
# 쿼리에서 직접 개수 지정
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="Brief summary of a cosmetic product",
    metadata_field_info=metadata_field_info,
    enable_limit=True,
)

results = retriever.invoke("2023년에 출시된 상품 1개를 추천해주세요")
print("\n쿼리에서 개수 지정 (1개):")
for doc in results:
    print(f"- {doc.page_content[:30]}... ({doc.metadata['year']}년)")


쿼리에서 개수 지정 (1개):
- 24시간 지속되는 매트한 피니시의 파운데이션 (2023년)


In [11]:
results = retriever.invoke("2023년에 출시된 상품 2개를 추천해주세요")
print("\n쿼리에서 개수 지정 (2개):")
for doc in results:
    print(f"- {doc.page_content[:30]}... ({doc.metadata['year']}년)")


쿼리에서 개수 지정 (2개):
- 24시간 지속되는 매트한 피니시의 파운데이션 (2023년)
- 비타민 C 함유 브라이트닝 크림 (2023년)


## 6. Query Constructor Chain 상세 분석

SelfQueryRetriever의 핵심인 Query Constructor가 어떻게 작동하는지 살펴봅니다.

In [12]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

# Query Constructor 프롬프트 생성
prompt = get_query_constructor_prompt(
    "Brief summary of a cosmetic product",
    metadata_field_info,
)

# 구조화된 쿼리 출력 파서
output_parser = StructuredQueryOutputParser.from_components()

# Query Constructor Chain 생성
query_constructor = prompt | llm | output_parser

In [13]:
# 프롬프트 내용 확인 (일부)
prompt_text = prompt.format(query="dummy question")
print("=== Query Constructor Prompt (일부) ===")
print("\n" + prompt_text[:400] + "\n....(중략)...\n")
print("Attributes:")
print("- category: 화장품 카테고리 ['스킨케어', '메이크업', '클렌징', '선케어']")
print("- year: 출시 연도")
print("- user_rating: 사용자 평점 (1-5)")

=== Query Constructor Prompt (일부) ===

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

...(중략)...

Attributes:
- category: 화장품 카테고리 ['스킨케어', '메이크업', '클렌징', '선케어']
- year: 출시 연도
- user_rating: 사용자 평점 (1-5)


In [14]:
# Query Constructor 테스트
query_output = query_constructor.invoke(
    {"query": "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"}
)

print("생성된 구조화 쿼리:")
print(f"- Query: {query_output.query}")
print("- Filter conditions:")
for condition in query_output.filter.arguments:
    print(f"  - {condition.attribute} {condition.comparator.value} {condition.value}")

생성된 구조화 쿼리:
- Query: 
- Filter conditions:
  - year >= 2023
  - user_rating >= 4.5
  - category == 스킨케어


## 7. 커스텀 Structured Query Translator 사용

특정 벡터 데이터베이스에 맞는 Query Translator를 사용하여 더 정밀한 제어가 가능합니다.

In [15]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

# 커스텀 Retriever 생성
retriever = SelfQueryRetriever(
    query_constructor=query_constructor,
    vectorstore=vectorstore,
    structured_query_translator=ChromaTranslator(),  # Chroma 전용 변환기
)

results = retriever.invoke(
    "2023년도에 출시한 상품 중 평점이 4.5 이상인 상품중에서 스킨케어 제품을 추천해주세요"
)

print("커스텀 검색 결과:")
for doc in results:
    meta = doc.metadata
    print(f"- {doc.page_content[:20]}... ({meta['category']}, {meta['year']}년, 평점: {meta['user_rating']})")

커스텀 검색 결과:
- 비타민 C 함유 브라이트닝 크림 (스킨케어, 2023년, 평점: 4.6)
- 수분 가득한 히알루론산 세럼 (스킨케어, 2024년, 평점: 4.7)


## 핵심 요약

### SelfQueryRetriever의 장점
1. **자동 필터 추출**: 자연어에서 메타데이터 필터를 자동으로 추출
2. **유연한 검색**: 의미적 유사성과 메타데이터 필터링을 동시에 수행
3. **복잡한 쿼리 지원**: AND, OR, NOT 등의 논리 연산자 자동 적용
4. **다양한 데이터 타입**: 문자열, 정수, 실수 등 다양한 메타데이터 타입 지원

### 최적화 팁
1. **명확한 메타데이터 설명**: AttributeInfo의 description을 구체적으로 작성
2. **적절한 LLM 선택**: 복잡한 쿼리에는 더 강력한 LLM 사용
3. **프롬프트 튜닝**: 필요시 query_constructor 프롬프트 커스터마이징
4. **벡터 DB 선택**: 사용 케이스에 맞는 벡터 데이터베이스와 translator 선택

### 사용 시 주의사항
- 메타데이터 필드가 많을수록 LLM 토큰 사용량 증가
- 복잡한 쿼리는 추가적인 LLM 호출 비용 발생
- 벡터 데이터베이스별로 지원하는 필터 연산자가 다를 수 있음